In [2]:
api_cols = [
    'B01001A_001E',
    'B01001_001E',
    'B01002_001E',
    'B19013_001E',
    'B19301_001E',
    'B23020_001E',
    'B25058_001E',
    'B25105_001E',
    'B25071_001E',
    'B24031_003E',
    'B24031_004E',
    'B24031_005E',
    'B24031_006E',
    'B24031_007E',
    'B24031_008E',
    'B24031_010E',
    'B24031_011E',
    'B24031_012E',
    'B24031_014E',
    'B24031_015E',
    'B24031_017E',
    'B24031_018E',
    'B24031_019E',
    'B24031_021E',
    'B24031_022E',
    'B24031_024E',
    'B24031_025E',
    'B24031_026E',
    'B24031_027E',
    'B12006_004E',
    'B12006_009E',
    'B12006_015E',
    'B12006_020E',
    'B12006_026E',
    'B12006_031E',
    'B12006_037E',
    'B12006_042E',
    'B12006_048E',
    'B12006_053E',
    'B12006_006E',
    'B12006_011E',
    'B12006_017E',
    'B12006_022E',
    'B12006_028E',
    'B12006_033E',
    'B12006_039E',
    'B12006_044E',
    'B12006_050E',
    'B12006_055E',
    'B15002_001E',
    'B15002_004E',
    'B15002_005E',
    'B15002_006E',
    'B15002_007E',
    'B15002_008E',
    'B15002_009E',
    'B15002_010E',
    'B15002_011E',
    'B15002_012E',
    'B15002_013E',
    'B15002_014E',
    'B15002_015E',
    'B15002_016E',
    'B15002_017E',
    'B15002_018E',
    'B15002_021E',
    'B15002_022E',
    'B15002_023E',
    'B15002_024E',
    'B15002_025E',
    'B15002_026E',
    'B15002_027E',
    'B15002_028E',
    'B15002_029E',
    'B15002_030E',
    'B15002_031E',
    'B15002_032E',
    'B15002_033E',
    'B15002_034E',
    'B15002_035E',
    'B15002_020E',
    'B15002_003E',
    'B08303_002E',
    'B08303_003E',
    'B08303_004E',
    'B08303_005E',
    'B08303_006E',
    'B08303_007E',
    'B08303_008E',
    'B08303_009E',
    'B08303_010E',
    'B08303_011E',
    'B08303_012E',
    'B08303_013E',
    'B08303_001E',
]

geo_cols = [
    'NAME',
    'GEO_ID',
    'STATE',
    'COUNTY',
    'REGION',
    'PLACE',
    'SUMLEVEL',
    'UA',
    'CBSA'
]



In [3]:
import os
import json
import requests
import pandas as pd

url = 'https://api.census.gov/data/{year}/acs/acs5?get={fields}&for={geo}:*&key={key}'
# geo = 'state'
# year = 2009

geos = [
    ('state','state'),
    ('county','county'),
    ('place','place'),
    ('metro', 'metropolitan statistical area/micropolitan statistical area')
]

years = list(range(2009,2021))
# years = [2021]

files = os.listdir('data')

with open('filters.json', 'r') as f:
    filters = json.load(f)

def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

# filtered = list(set(api_cols) - set(filters.get(str(year),[])))
# field_groups = [geo_cols + x for x in list(divide_chunks(filtered,40))]


for year in years:
    filtered = list(set(api_cols) - set(filters.get(str(year),[])))
    field_groups = [geo_cols + x for x in list(divide_chunks(filtered,40))]
    for geo in geos[3:]:
        if f'{geo[0]}_{year}_census_api.csv' not in files:
            base_df = None
            for group in field_groups:
                count = 0
                error = True
                while error and count < 200:
                    error = False
                    print(f'attempt #{count}')
                    count += 1
                    key = 'c8763b1f3e686c707cf581321b8c16152e9d30e3'
                    filtered_cols = list(set(group) - set(filters.get(str(year),[])))
                    fields = ','.join(filtered_cols)
                    formatted_url = url.format(year=year, fields=fields, geo=geo[1], key=key)

                    x = requests.get(formatted_url)



                    if x.status_code == 400 and 'error: error: unknown variable ' in x.text:
                        bad_var = x.text.replace('error: error: unknown variable ','').replace("'",'')
                        filters[str(year)] = filters.get(str(year),[]) + [bad_var]
                        with open('filters.json', 'w') as f:
                            json.dump(filters, f, indent=4)


                    # print(formatted_url)
                    # print(x.status_code)
                    
                    if x.status_code == 200:
                        try:
                            data = x.json()
                        except:
                            error = True
                            print(f'failed to parse json')
                    else:
                        error = True
                        print(f'failed with code: {x.status_code}')

                    # print(x.text)

                tmp_df = pd.DataFrame(data[1:],columns=data[0],).set_index('GEO_ID')
                if base_df is None:
                    base_df = tmp_df
                else:
                    cols_to_use = tmp_df.columns.difference(base_df.columns)
                    base_df = base_df.join(tmp_df[cols_to_use])
                # print(base_df.head())

            base_df['year'] = year
            base_df['geo_type'] = geo[0]
            base_df.to_csv(f'data/{geo[0]}_{year}_census_api.csv')
    

attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0
attempt #0


In [6]:
import os
import pandas as pd
files = os.listdir('data')
print(files)
exp_len = len(geo_cols) + len(api_cols)

exp_cols = set(geo_cols + api_cols)

for file in files:
    if 'processed' not in file and 'backup' not in file:
        df = pd.read_csv('data/' + file, nrows = 5)
        # if len(df.columns) < exp_len:
        # print(file)
        # print(file+':',len(df.columns),'/',exp_len)
        diff = exp_cols - set(df.columns)
        if diff:
            print(file)
            print(diff)


['metro_2018_census_api.csv', 'county_2016_census_api.csv', 'processed', 'metro_2012_census_api.csv', 'state_2009_census_api.csv', 'metro_2014_census_api.csv', 'state_2010_census_api.csv', 'backup', 'place_2015_census_api.csv', 'place_2020_census_api.csv', 'state_2015_census_api.csv', 'state_2017_census_api.csv', 'county_2014_census_api.csv', 'metro_2011_census_api.csv', 'metro_2009_census_api.csv', 'state_2013_census_api.csv', 'place_2017_census_api.csv', 'county_2017_census_api.csv', 'metro_2020_census_api.csv', 'metro_2010_census_api.csv', 'place_2009_census_api.csv', 'place_2018_census_api.csv', 'metro_2015_census_api.csv', 'state_2019_census_api.csv', 'metro_2016_census_api.csv', 'place_2014_census_api.csv', 'state_2011_census_api.csv', 'state_2016_census_api.csv', 'county_2012_census_api.csv', 'place_2011_census_api.csv', 'county_2013_census_api.csv', 'metro_2019_census_api.csv', 'county_2010_census_api.csv', 'county_2015_census_api.csv', 'place_2019_census_api.csv', 'metro_2013_